<a href="https://colab.research.google.com/github/Pushkaran-P/Research-Paper-Search-Engine/blob/main/2_Creating_The_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Libraries Needed

!pip install chromadb sentence-transformers -q
!pip install langchain -q
!pip install "accelerate>=0.16.0,<1" -q
!pip install "transformers[torch]>=4.28.1,<5" -q
!pip install "torch>=1.13.1,<2" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

Restart session after installing for the accelerate library to be properly installed

## Database

In [1]:
# Imports

import pandas as pd
import chromadb
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

In [27]:
# Loading dataframe and combining title and abstract

df = pd.read_csv('database.csv')

df['abstract'] = df['abstract'].fillna(' No Abstract Available ')
df['combined'] = 'The title of this research paper is ' + df['title'] + '. The abstract of this research paper: ' + df['abstract']
df['Id'] = df['Id'].astype(str)

In [3]:
# ChromaDB expects a list of documents
documents = df["combined"].tolist()
# ChromaDB expects a List of dicitonaries for its metadata
metadatas = [{ 'year': row['year'], 'authors': row['authors'], 'citations': row['citations'], 'original_id': row['Id'] } for _, row in df.iterrows()]
# ChromaDB expects a list of ids for its ids
ids = df["Id"].tolist()

In [4]:
# Initializing the Huggingface embeddings here

model_name = "BAAI/bge-base-en-v1.5"
# Device can be changed to "cuda:0" or "cpu"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Creating Chroma db here
db = Chroma.from_texts(texts = documents, collection_name = 'research_db',
                       embedding = embeddings, persist_directory = "/content/research_db",
                       metadatas = metadatas,
                       ids = ids)

In [6]:
import os
import zipfile

def zip_folder(folder_path, zip_name):
    # Create a ZipFile object
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Iterate over all the files in the folder
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Write each file into the zip file
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(folder_path, '..')))

# Use the function to zip the folder
zip_folder('/content/research_db', 'research_db.zip')

## Other

In [7]:
# Check if data is inserted correctly
print("There are", db._collection.count(), "in the collection")
# Initialize retriever (very imp)
retriever = db.as_retriever()

There are 601 in the collection


In [8]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Download and use the llm engine

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map = "auto")
llm = HuggingFacePipeline(pipeline=generate_text)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## L1

In [9]:
from langchain import hub
!pip install langchainhub -q

In [10]:
prompt = hub.pull("rlm/rag-prompt")

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    #{"context": retriever | format_docs, "question": RunnablePassthrough()}
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
rag_chain.invoke("Give some machine learning papers?")

'Subjective Answers Evaluation Using Machine Learning and Natural Language Processing \nxCorrection to “Classification of Poetry Text Into the Emotional States Using Deep Learning Technique” \nReviewer Recommendations Using Document Vector Embeddings and a Publisher Database: Implementation and Evaluation'

## L1 advanced

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [21]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "Suggest some machine learning papers?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg)
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "Year in which first paper was published?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Machine Learning: 
- Mahaseni, S., Fernandes, V., Cabral, I., et al. “Semi-supervised learning of topic models with latent Dirichlet allocation for EMR text classification.” Big data in healthcare 4, no. 4 (2014): 541.
- Wu, T. M. N., Del Vecchio, S., Hu, T., et al. “WordNet based global term analysis for document representation learning.” Advances in neural information processing systems 21.3 (2009): 711-720.

Natural Language Processing:
- Wang, S., Su, D., Hu, T. “Using semantic roles to identify medical facts in unstructured text.” Data Engineering & Knowledge Engineering Review 21, no. 2 (2009): 123-140.
- Aprikian, N., Tan, C., and Yih, I. L. “Improving sentiment analysis of free-form text using named entity recognition.” Proceedings of the Annual Conference of the North American Chapter of the Association for Computational Linguistics (2006).


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Machine Learning: 2009'

In [ ]:
chat_history = []

while True:
    question = input("Input Prompt: ")
    if question.lower() == 'exit':
        print('Exiting')
        sys.exit()
    if question == '':
        continue
    ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
    print(ai_msg)
    chat_history.extend([HumanMessage(content=question), ai_msg])